In [106]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['introelapse']>3000].index.to_list())
train = train.drop(train[train['testelapse']>10000].index.to_list())
train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])
# train.drop('hand',axis=1,inplace=True)
# train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])
# test.drop('hand',axis=1,inplace=True)
# test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
test_fill_na = test.fillna(test.mean())

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from string import ascii_lowercase
from itertools import combinations

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [9]:
test = pd.read_csv('./data/test.csv').drop('index',axis=1)
test['TIPI3'].value_counts()

3.0    15360
4.0     9364
5.0     6188
2.0     2875
1.0     1502
Name: TIPI3, dtype: int64

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['introelapse']>3000].index.to_list())
train = train.drop(train[train['testelapse']>10000].index.to_list())
train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])
train.drop('hand',axis=1,inplace=True)
train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])
test.drop('hand',axis=1,inplace=True)
test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
test_fill_na = test.fillna(test.mean())

In [3]:
from pycaret.classification import *

clf = setup(data = train, target = 'nerdiness')

,Description,Value
0,Session id,1865
1,Target,nerdiness
2,Target type,Binary
3,Original data shape,"(14961, 74)"
4,Transformed data shape,"(14961, 74)"
5,Transformed train set shape,"(10472, 74)"
6,Transformed test set shape,"(4489, 74)"
7,Numeric features,73
8,Rows with missing values,14.7%
9,Preprocess,True


In [4]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7674,0.8580,0.8261,0.7703,0.7972,0.5252,0.5271,0.1390
rf,Random Forest Classifier,0.7683,0.8541,0.8254,0.7720,0.7978,0.5273,0.5290,0.1380
catboost,CatBoost Classifier,0.7587,0.8296,0.8200,0.7622,0.7899,0.5073,0.5095,9.8440
lightgbm,Light Gradient Boosting Machine,0.7520,0.8225,0.8114,0.7578,0.7836,0.4940,0.4957,0.1380
gbc,Gradient Boosting Classifier,0.7319,0.8035,0.7990,0.7383,0.7674,0.4521,0.4542,0.3790
lda,Linear Discriminant Analysis,0.7245,0.7929,0.8028,0.7278,0.7634,0.4356,0.4388,0.1040
ada,Ada Boost Classifier,0.7195,0.7893,0.7769,0.7328,0.7541,0.4284,0.4295,0.1560
lr,Logistic Regression,0.7099,0.7778,0.7833,0.7182,0.7492,0.4066,0.4089,0.2980
nb,Naive Bayes,0.6896,0.7519,0.7465,0.7089,0.7270,0.3677,0.3686,0.1090
qda,Quadratic Discriminant Analysis,0.6635,0.7194,0.7850,0.6672,0.7209,0.3042,0.3111,0.1060


Processing:   0%|          | 0/67 [00:00<?, ?it/s]

In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
predictions

In [ ]:
index = test_index

submission = pd.DataFrame({
    "index" : index,
    "nerdiness" : predictions['Score']
})
submission.to_csv('./data/model_auto_ml.csv', index=False)

In [ ]:
y_train = train_fill_na['nerdiness']
x_train = train_fill_na.drop('nerdiness',axis=1)

In [ ]:
test = test_fill_na

In [ ]:
k_fold = KFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
from catboost import CatBoostClassifier

params = {'bagging_temperature': 0.375906, 'depth': 9.0, 'l2_leaf_reg': 68.8, 'learning_rate': 0.011, 'subsample': 0.76046}

clf1 = CatBoostClassifier(**params, iterations=5000, eval_metric='AUC',allow_writing_files=False,od_type='Iter',random_state=777)
clf2 = LGBMClassifier()
clf3 = GradientBoostingClassifier()
soft_vote  = VotingClassifier([('r1',clf1), ('r2', clf2), ('r3',clf3)], voting='soft')
soft_vote.fit(x_train, y_train)

In [ ]:
model = soft_vote
pred_y = model.predict_proba(test_fill_na)
pred_y = pred_y[:,1]

index = test_index

submission = pd.DataFrame({
    "index" : index,
    "voted" : pred_y
})
submission.to_csv('./data/model1.csv', index=False)

# model2

In [154]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['introelapse']>3000].index.to_list())
train = train.drop(train[train['testelapse']>10000].index.to_list())
train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])
train.drop('hand',axis=1,inplace=True)
# train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])
test.drop('hand',axis=1,inplace=True)
# test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
test_fill_na = test.fillna(test.mean())

In [155]:
y_train = train_fill_na['nerdiness']
x_train = train_fill_na.drop('nerdiness',axis=1)

In [156]:
test = test_fill_na

In [157]:
import pandas as pd
import numpy as np

from string import ascii_lowercase
from itertools import combinations

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [158]:
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
import eli5
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt

import warnings
import gc
warnings.filterwarnings("ignore")

In [159]:
def lgbm_rfe_777(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=777)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [160]:
lgbm_archive_4040 = lgbm_rfe_777(x_train, y_train)

77 0.847859879109879
75 0.8525857588357587
73 0.8493165641603142
71 0.8494961307461307
69 0.8596848049973052
67 0.8510740307615308
65 0.8558128440940942
63 0.8584522022022021
61 0.8550867574305074
59 0.8587358392045893
57 0.8570559742434742
55 0.8574268378955879
53 0.856757598945099
51 0.8526835128397628
49 0.8449901223338723
47 0.8506691787941788
45 0.8528841341341341
43 0.8561040006352506
41 0.8539507055132054
39 0.8468227843227842
38 0.8514451951951951
37 0.8468715109340109
36 0.8549065892815894
35 0.8482054771117271
34 0.8481224614037115
33 0.8503112487487487
32 0.8472234253484254
31 0.8476168476168476
30 0.8451928490990991


In [161]:
model = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_1 = x_train[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]]

model.fit(x_train_1, y_train)

pred_y1 = model.predict_proba(test[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]])
pred_y1 = pred_y1[:,1]

In [164]:
def lgbm_rfe_1234(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1277)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [165]:
lgbm_archive_1234 = lgbm_rfe_1234(x_train, y_train)

77 0.851471383121075
75 0.8509694044937102
73 0.8521408902867136
71 0.8495800102034592
69 0.8503599890863655
67 0.8529248145880937
65 0.8523703315456541
63 0.8529345263874141
61 0.8531144981685671
59 0.8503675764295844
57 0.8484188431972336
55 0.8490479856969475
53 0.8427477593816739
51 0.8517927829798287
49 0.8504613559917704
47 0.8505041486075253
45 0.8481966857877832
43 0.8489526886661178
41 0.8465930249250295
39 0.8468904487792117
38 0.8491745425818396
37 0.8476400782892423
36 0.8484036685107957
35 0.8412985768268577
34 0.8482525286338746
33 0.8411365111757013
32 0.8461444611939261
31 0.8349743745070124
30 0.8461101664025765


In [166]:
model2 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_2 = x_train[lgbm_archive_1234.iloc[lgbm_archive_1234[lgbm_archive_1234['score']==lgbm_archive_1234['score'].max()].index[0],2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[lgbm_archive_1234.iloc[lgbm_archive_1234[lgbm_archive_1234['score']==lgbm_archive_1234['score'].max()].index[0],2]])
pred_y2 = pred_y2[:,1]

In [167]:
def lgbm_rfe_99087(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=9977)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [168]:
lgbm_archive_99087 = lgbm_rfe_99087(x_train, y_train)

77 0.8567396494774387
75 0.8556642546573995
73 0.8550942099571064
71 0.8547054321475915
69 0.8519842926441041
67 0.8545031090018236
65 0.8543480861390029
63 0.8508155179106336
61 0.857259037100511
59 0.8532980198575744
57 0.8467550174919497
55 0.8492607118356904
53 0.8476372440596948
51 0.8527697825041441
49 0.8522720614622928
47 0.8488697978843652
45 0.8461745972457198
43 0.8501017413828038
41 0.84628109917313
39 0.8548036946708755
38 0.8513269712926953
37 0.8544234614286026
36 0.8490925669588908
35 0.8492970262464692
34 0.8498371649614153
33 0.8491914398084064
32 0.8495994428942159
31 0.8467336560738445
30 0.8448025838771341


In [169]:
model3 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_3 = x_train[lgbm_archive_99087.iloc[lgbm_archive_99087[lgbm_archive_99087['score']==lgbm_archive_99087['score'].max()].index[0],2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[lgbm_archive_99087.iloc[lgbm_archive_99087[lgbm_archive_99087['score']==lgbm_archive_99087['score'].max()].index[0],2]])
pred_y3 = pred_y3[:,1]

In [170]:
def lgbm_rfe_42(x_data, y_data, ratio=0.975, min_feats=30):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**3)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive


In [171]:
lgbm_archive_42 = lgbm_rfe_42(x_train, y_train)

77 0.8519725097102603
75 0.8544796868591831
73 0.847738293406673
71 0.8567299341391039
69 0.8512858555885262
67 0.8524073604979374
65 0.851249366722154
63 0.8531018552025283
61 0.8567938650454754
59 0.8471457262792212
57 0.8491037610672842
55 0.8542293913295216
53 0.8533587850715303
51 0.8504203758654797
49 0.8510554630768861
47 0.8542839738486405
45 0.8513135991894043
43 0.8452410677667608
41 0.8529221249185786
39 0.8453342500784059
38 0.8429247786543146
37 0.84436352560855
36 0.8482419000747871
35 0.8460013027429977
34 0.8479355142216112
33 0.8519655738100407
32 0.8425499384815806
31 0.8481891269209427
30 0.8478338882053509


In [172]:
model4 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_4 = x_train[lgbm_archive_42.iloc[lgbm_archive_42[lgbm_archive_42['score']==lgbm_archive_42['score'].max()].index[0],2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[lgbm_archive_42.iloc[lgbm_archive_42[lgbm_archive_42['score']==lgbm_archive_42['score'].max()].index[0],2]])
pred_y4 = pred_y4[:,1]

In [173]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)

index = pd.read_csv('./data/test.csv').index.to_list()

submission = pd.DataFrame({
    "nerdiness" : index,
    "voted" : pred_all
})
submission.to_csv('./data/model2_2.csv', index=False)

# model3

In [174]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)
# train.drop('introelapse',axis=1,inplace=True)
# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['introelapse']>3000].index.to_list())
train = train.drop(train[train['testelapse']>10000].index.to_list())
train = train.drop(train[train['surveyelapse']>10000].index.to_list()) # 나에 대한 추가 질문

test = pd.read_csv('./data/test.csv').drop('index',axis=1)
# test.drop('introelapse',axis=1,inplace=True)
test.loc[test['familysize']>100,'familysize']= train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()
test.loc[test['introelapse']>3000,'introelapse']= train['introelapse'].mean()
test.loc[test['testelapse']>10000,'testelapse']= train['testelapse'].mean()
test.loc[test['surveyelapse']>10000,'surveyelapse']= train['surveyelapse'].mean()

test_index = pd.read_csv('./data/test.csv')['index']

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] - train['Q3'] - train['Q6'] - train['Q7'] - train['Q10'] + train['Q12'] + train['Q15'] - train['Q16']
train['V'] = -train['Q4'] + train['Q5'] + train['Q8'] - train['Q11'] + train['Q13'] - train['Q14'] - train['Q17'] + train['Q18'] + train['Q20']
train['M'] = -train['Q9'] + train['Q19']
train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])
train.drop('hand',axis=1,inplace=True)
# train.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# train.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# train.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# train.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
train_fill_na = train.fillna(train.mean())


value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))
# rank = list(map(lambda x : 1 if x>2000 else (2 if x>1000 else (3 if x>500 else (4 if x>200 else (5 if x>100 else 6)))),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] - test['Q3'] - test['Q6'] - test['Q7'] - test['Q10'] + test['Q12'] + test['Q15'] - test['Q16']
test['V'] = -test['Q4'] + test['Q5'] + test['Q8'] - test['Q11'] + test['Q13'] - test['Q14'] - test['Q17'] + test['Q18'] + test['Q20']
test['M'] = -test['Q9'] + test['Q19']

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])
test.drop('hand',axis=1,inplace=True)
# test.drop(['VCL7','VCL8','VCL11'],axis=1,inplace=True)
# test.drop([('TIPI'+str(i)) for i in range(1,11)],axis=1,inplace=True)
# test.drop([('Q'+str(i)) for i in range(1,20)],axis=1,inplace=True)

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : x.mean(),axis=1)
# test.drop([('Q'+str(i)) for i in range(20,27)],axis=1,inplace=True)
test_fill_na = test.fillna(test.mean())

In [175]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

y_train = train_fill_na['nerdiness']
train_x = train_fill_na.drop('nerdiness',axis=1)
test_x = test_fill_na.copy()

ss.fit(train_x)
train_x = ss.transform(train_x)
test_x = ss.transform(test_x)
# train_fill_na

In [179]:
len(train_x[0])

77

In [180]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Used',DEVICE)



train_y = y_train.copy()

# train_y = 2 - train_y.to_numpy()
# train_x = train_x.to_numpy()
# test_x = test_x.to_numpy()

train_y_t = torch.tensor(train_y, dtype=torch.float32)
train_x_t = torch.tensor(train_x, dtype=torch.float32)
test_x_t = torch.tensor(test_x, dtype=torch.float32)

train_x_t[:, :26] = (train_x_t[:, :26] - 3.) / 2.
test_x_t[:, :26] = (test_x_t[:, :26] - 3.) / 2

train_x_t[:, 30:40] = (train_x_t[:, 30:40] - 3.) / 2.
test_x_t[:, 30:40] = (test_x_t[:, 30:40] - 3.) / 2

test_len = len(test_x_t)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 48
BATCH_SIZE = 1024
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4, # in colab use 2
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(77, 180, bias=False),
            nn.LeakyReLU(0.5, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(180, 32, bias=False),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
            # nn.Linear(128, 32, bias=False),
            # nn.ReLU(inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid()
        ).to(DEVICE)
        # criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        criterion = torch.nn.BCELoss()

        optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=4e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    # running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                    running_acc += (((pred) >
                                    0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'.format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = ((model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))

df = pd.read_csv('./data/sample_submission.csv')
df.iloc[:, 1:] = prediction

Used cuda:0


07/07: 100%|██████████| 48/48 [00:39<00:00,  1.21it/s]


R1 -> 0.6530


07/07: 100%|██████████| 48/48 [00:39<00:00,  1.21it/s]


R2 -> 0.6536


...

R3 -> 0.6527


07/07: 100%|██████████| 48/48 [00:39<00:00,  1.22it/s]


R4 -> 0.6489


07/07: 100%|██████████| 48/48 [00:39<00:00,  1.21it/s]

R5 -> 0.6569


In [ ]:
for i in df['nerdiness']:
    print(i)

In [ ]:
df.to_csv('./data/model3.csv', index=False)

# final Ensemble

In [ ]:
model1 = pd.read_csv('./data/model1.csv', index_col = 'index')
model2 = pd.read_csv('./data/model2.csv', index_col='index')

pred_y = (model1)*(0.7) + (model2)*(0.3)

test = pd.read_csv('./data/test.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'voted': pred_y['voted']
    })

submission.to_csv('./data/combined_model1_model2.csv', index=False)

In [ ]:
combined_12 = pd.read_csv('./data/combined_model1_model2.csv', index_col = 'index')
model3 = pd.read_csv('./data/model3.csv', index_col='index')
model3['voted'] = model3['voted']-1

In [ ]:
pred_y = (model3)*(0.8) + (combined_12)*(0.2)

test = pd.read_csv('./data/test_x.csv')
index = test['index']

submission = pd.DataFrame({
    'index': index,
    'voted': pred_y['voted']
    })

submission.to_csv('./data/submission_final_fix_lgbm.csv', index=False)